# 실습 - 네이버 로그인 및 카페 크롤링

### 1. 로그인 계정 설정

In [8]:
# naverid = "네이버 로그인 아이디 입력"
# naverpw = "네이버 로그인 비밀번호 입력"

### 2.크롬 브라우저 설정

In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [3]:
dirver_loc = "d:/chromedriver/chromedriver.exe"

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
# 헤드리스 설정 시 리눅스 같이 웹화면 없는경우에도 웹페이지 실행가능함
#options.add_argument('headless')
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
# 웹 드라이버 정의
driver = webdriver.Chrome(dirver_loc, options=options)

### 3. 네이버 로그인

##### 1. 네이버 페이지 이동

In [4]:
import pyperclip

naver_page= 'https://www.naver.com/'

driver.get(naver_page)
time.sleep(2)

# 로그인 버튼을 찾고 클릭합니다
loginBtn = '//*[@id="account"]/a'
login_btn = driver.find_element_by_xpath(loginBtn)
login_btn.click()
time.sleep(2)

# id, pw 입력할 곳을 찾습니다.
idTextBox = '//*[@id="id"]'
pwTextBox = '//*[@id="pw"]'
tag_id = driver.find_element_by_xpath(idTextBox)
tag_pw = driver.find_element_by_xpath(pwTextBox)
tag_id.clear()
time.sleep(1)

# id 입력
tag_id.click()
# 클립보드 생성 복사
pyperclip.copy(naverid)
tag_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# pw 입력
tag_pw.click()
pyperclip.copy(naverpw)
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 로그인 버튼을 클릭합니다
loginBtn = '//*[@id="log.login"]'
login_btn = driver.find_element_by_xpath(loginBtn)
login_btn.click()

### 4. 카페 접속

In [5]:
time.sleep(2)
keyword = "오류"

#카페 접속 및 검색어 크롤링
cafe_url = 'https://cafe.naver.com/playbattlegrounds'
#키워드 등록
keyword_list = [
    '오류'
]
driver.get(cafe_url)
time.sleep(2)

### 5. 검색어 타입 및 프레임 전환

In [6]:
from selenium.webdriver.common.keys import Keys

#driver.get_screenshot_as_file('naver_main_headless.png')

# 상단 우측 검색어입력 버튼 
search_input=driver.find_element_by_xpath('//*[@id="topLayerQueryInput"]')

#데이터 크롤링
search_input.send_keys(keyword)
search_input.send_keys(Keys.ENTER)

# 프레임 살펴보기
iframes = driver.find_elements_by_css_selector('iframe')
for iframe in iframes:
    print(iframe.get_attribute('name'))

driver.switch_to_default_content # 상위프레임으로 전환
driver.switch_to.frame("cafe_main") # 특정 프레임으로 전환

time.sleep(0.5)
html = driver.page_source



padding
padding
padding
padding

cafe_main
cafe_check

checkFrame



### 6. 페이지 크롤링

In [7]:
import bs4
import pandas as pd
bs = bs4.BeautifulSoup(html, "html.parser")

boardDiv = bs.find("div",attrs={"class":"article-board result-board m-tcol-c"})
boardTrs = boardDiv.find(name="table").find(name="tbody").findAll("tr")

rowList = []
columnList = []
for i in range(0, len(boardTrs)):
    if i%2!=0:
        continue
    rowNumber = boardTrs[i].find(name="div").find("div").text
    columnList.append(rowNumber)
    title = boardTrs[i].find(name="a").text.replace("\n","").replace("  ","")
    columnList.append(title)
    tds = boardTrs[i].findAll("td")
    for j in range(0, len(tds)):
        if (j==0) | (j==1):
            continue
        columnList.append(tds[j].text.replace("  "," ").replace("\n",""))
    rowList.append(columnList)
    columnList = []

pd.DataFrame(rowList, columns=["번호","제목","작성자","시간","조회수"])

,번호,제목,작성자,시간,조회수
0,4598623,핵신고,qkfxkssmre,20:07,8
1,4598579,[스팀]🐣신생클 YTS에서 가족을 찾아요🐣,YTS Zero,19:51,5
2,4598412,[신생클] 🤗가입이 이렇게 간편하다고!?🤗,YTS Zero,18:43,9
3,4598401,핵신고,핵쟁이꺼저,18:31,23
4,4598344,배틀아이 오류(?),Luan2006,17:55,56
5,4598251,😊신생클 YTS CLAN 에서 클랜원을 모집해요😊,YTS Zero,17:07,11
6,4598189,핵신고,핵쟁이꺼저,16:39,17
7,4598106,핵몇명,테닉,16:01,15
8,4597778,스팀 핵 제보,pendol13,12:58,25
9,4597509,🐥신생클 YTS 가입이 이렇게 간단해도 되는 거야..?🐥,YTS Zero,08:52,12
